# Задание 1. Сравнение предложений

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:
* кошки (животные)
* UNIX-утилита cat для вывода содержимого файлов
* версии операционной системы OS X, названные в честь семейства кошачьих

Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

Выполните следующие шаги:
1. Скачайте файл с предложениями (sentences.txt).
2. Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
3. Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.
4. Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
5. Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.
6. Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
7. Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).

In [1]:
import numpy as np
import re

### Шаги 1-4. Парсинг предложений из текста, токенизация на слова, создание списка всех слов, встречающихся в предложениях

In [2]:
sentences = []
words_dict = {}
i = 0
with open('sentences.txt', 'r') as f:
    for line in f:
        text = line.lower()
        words = re.split('[^a-z]+', text)[:-1]
        sentences.append(words)
        for word in words:
            if word not in words_dict:
                words_dict[word] = i
                i += 1

### Шаг 5. Создание матрицы размера n * d, где n — число предложений. Заполнение: элемент с индексом (i, j) в этой матрице равен количеству вхождений j-го слова в i-е предложение

In [3]:
matrix = []
for sentence in sentences:
    vector = np.zeros(len(words_dict))
    for word in sentence:
        vector[words_dict[word]] += 1
    matrix.append(vector)

matrix = np.array(matrix)
print(matrix)
print(f'Matrix shape: {matrix.shape}')

[[1. 1. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 2. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 1. 1.]]
Matrix shape: (22, 254)


### Шаг 6. Расчет косинусного расстояния между первым предложением и остальными

In [5]:
from scipy.spatial.distance import cosine

cosine_list = []
for i in range(1, matrix.__len__()):
    cosine_list.append((cosine(matrix[0], matrix[i]), i))
cosine_list.sort()

### Шаг 7. Сохранение результата в файл

In [6]:
with open('answer1.txt', 'w') as f:
    answer = ' '.join([str(i[1]) for i in cosine_list[:2]])
    f.write(answer)